<a target="_blank" href="https://colab.research.google.com/github/gse-unil/2024_Geoinformatique_I/blob/main/Exercices-SIG/w10-2-selections-queries-joins/2-tuto-geopandas.ipynb">
  <img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHUAAAAUCAYAAACzrHJDAAAABGdBTUEAALGPC/xhBQAAACBjSFJNAAB6JgAAgIQAAPoAAACA6AAAdTAAAOpgAAA6mAAAF3CculE8AAAABmJLR0QA/wD/AP+gvaeTAAAACXBIWXMAAA7DAAAOwwHHb6hkAAAAB3RJTUUH6AsLEC4F8VnbjAAACgpJREFUaN7tmXl0VOUZxn93mTWZzCQzZJksQEISVlmCRxGp4FJt6xIUl8PiBu05teoftVY9tdvpplatdnEBPT1a0Z4jcLRS1CqigAhBIAKCgCRmISHJbMlk9rn36x8TkoyZRMQaTlvec+6Zmfve733eeZ/veb977yclEgmxdu1atm3bRiAQYLRsfeX3Rg3r/8EKso1cP20M980fi7pmzRo2bNgw6kkIIU53Hf6n7HgwxuPbWgFQt27deloKfIbUr8de/KgDdTRb7mA7Q+rXYx3BGOrJFDfPEmacw4tJ1WjtcdDS7Ujzu209lDn8yJKgOeCgtcfxhTHPkPr1mTqSM8cUZdmMD6lxtyAxQEKD38nDWy+k0NbDkrN2UZHnSRt3yFPAUzvnEIhaTympXIuK3axyPBgnmtRHrRhFNiNlDjM7WnpOabzNpOC0Gk4671nFNrpCCVoC0RGvu2KSiw2HvGj6yQlhWKXmyFFumrKTWe7WIT6DrFHm8PLD897DqGhD/NWuDm5w7+bFxll065aM8TPhOiwqT9ZOZKbbRksgwvg8K6vqjvHQ5iZGQ9jVuUaumJzL9ubuLzXOZlJ5/Ioq5pc7OOoNU+Kw8FF7kBVrD9Ib04Ydt3zmGLZ8FuAlf2TE+CsXVjP+ofdJal+R1KWOXYw9FuCQIZ9ce5j3GiuIJFRq3K28cnAqK2bX9RPa2m3ng5ZxyJLgG+MaaGvMoTLk4Xp7PU/7zs0YPxPuqqsn4vf5mPTA8yQkA0XZBt64fzGecIJn6o6lXVtkM9HRG0f/XBybScWsynSF4kPiGxUZp9VAezCWMaeNB1p4u64eOb98CJY3nCCuZVbfqqurifQGqb5jNRGhQjLGtedPxySZCPbl57CoyJKEL5wYqEE8gohF+mthNSq4+pQ+GEsIgdB1CrKN9Ma1ESfKsKS6DUEqjamWamzQ+KnnUiLJVKd+++gEql1dFGQHATjscfHglgUkNBmANw9X8mPXZiQFJpk6cclBurTsLyR1XK6F+eW5VDy5Aa10OjISHck4v1q7lfsWzmXVjlYWzyjkpho3iiwRjiUZ77Sy6IW9HOoKYTHIPLVwEhOcVnpjSXTgutX7CMaSrFs2nebuKFPzs3BYVBp8Ua5/ce8Q9V8+q5zFM+ay+KV9XDOtgFtnu4lrOmZFosKVxZXP1fNJZyhtTIndzMUTnFTcvYGYexqylKrDmoNeJKsdk8nIX6+dwvhcM0ld4AknWPr3/YTiGkL0ESYEd8wtY9G0fLqCMSYX2vjZv46yZl9HP86jl1fjzjExwZXFyh2tPLa1+cuRWmII9H+vCxUTTigwaE0dvIZuaqggnpT6/ZGkQl24hG/ZDgFQZgjQmcz6QlIrHSpNnQH85oK+UAIUA3u8MSrGZKNIICI9TB5TztR7nscXinPz3EoeuHQWtX/byw/mlOD1elj68Fsgyfxy0XncPqeY3276DBHrhUiEBT9fi4LOrt/dxPTCbPa0BdNy0CNBRDQ7VehwDxOd45nxk9UEQlHuvmwGK2pKuGtDQ9qYqlyVZk8An7kQkOifKdl5CGD5bDdqIsy5965GSPD8bd/h9jnFPPBuU1/NUqSufHMnj73UCYpKYZbCB7+5mZf3Hu/H2XmwgZVv1+MwQv3vl/Pqx500+DK37YykymJA3pqQhxBgVJIDAWRtiD+kKQOxJD1jq/38OZGIgSQhZIXBEpJMlr7rdQSw+UAz/uwyJLvKq0cCPLjEjhCCBWOz8QYVfr28FoBxWQKryZDCEfD63iak4snoCA62enBbYXfGvPrUA2w/3Eq3rRTJrvKxL8HZ1XKGvONIkjwk7xN2TpGJtTuPIEqmABIv7zzKigVn9ccRfbVwl5Tyo6UXUOWyYlYlnFlGHGYVfyTVrl/Z34Hknki3nmTLgSbOLrJy1Bs+eVLbEwPtcpa1jde7K9CR+s8d68np/37+2EbeOTpugEQENdb2/t9t8eyTIvVgRw9jXXbyrCre0MC6M7PEzpF2HwlNT9VMVgcKmOXgxEyXhcbmT1qpa/Kl4icTBMNRhFwAQFyxIERqLdWQkPXEUIJOFFqkFBTTJYSUwtJkA3IfVlrex3soddpwWg14MqzjQkuCKRshgikKzTbQkoOwUngv3DCNh16r4/vbD6JpGp1P34ZBGiQYqwMR0UFSQDGgx33DPhbKJ3r64KMhaqctYUNIEHdL3H/hu5xT2sKMonZuqdlNa3c2gYgZgN5mE9917WKapYOZljbuzN/BeKMfgNa4jeZYDpkwPn809Whs3NfIw9+uwiBLqdmbY+L+iyt54q36PhEI5k10k2tREUJQO6WAbYdaEUKwcX8Tc6pLqfcKdnt09gQUfEbXIEVI/Vhpikw76J8kgwk+odzBvhNHa0jjjT2f8viV1VhUuf/8dWcV4LQaeP9wG9fMLO2ThGDRzFK2Hj42CCP1WeYwsaUlhF5UzWUXXYDdakLoA12udmoBQggcZpV5VUXsaPQMW8uMShXAs13TWTJjHzUTUqqrHuPt97ttQZ7YMZtLcxoojwXAAtMtHWkxEkLhBe/UIXenwykV1cjNqzaycsUlHL5nHs2BKOMcZv78eh3P7DoOtnyEgI+bO1m7bAbRpE6Z3UjtI68gyOeRd47wxGIn9fdewmf+CKUOM3/Z0siqnW1phAzGH6JUBhP5uTz72viQvBUjtzz7Ln+8cT6H7plHgzdMqcNMXUMX6/e388x2D/Oqiqi781ySuqCty8+f3vsUYc1Pm1x/WL+DzXd9kyOeMIHeMB3dobQcawot/PPWGqrGWHn0tQ9oCAKGzLWVFi5cOOzDjzsnyO3n7aI8z592vjtq5q71F1Es93Kjaz95avqC3ZnM4rmuqTTEHMOFzrxLoycRnY3Y5Ti5djvHOj0ksscg5RYDsHRyDvPH21m+8k2Kcm10eP2IgkowWkAIRFcjFj1KvtNBh7ebqNWJlJOPaD+EZC8Eqz1VyONHkGwuyMpNxw/5EUEPUmElotcLIT9SwYSULxxAdHciFVVlyFtDeJrI0kKMcebR7vESky1I+eUgyYiOT3EYBLIi44toqZiyguhsAEsOks2F8B/DKUJoipFANAmRHqTSaaAYEA0fQnYedpOMFg3Ta8pFchQNW1uptrZ2xCdaSYIql5dKlx+TkqSlO4f6tgLifTdDiqQzyeylzBREAM2xHD6J5KEhjxR25K03XQMtCQYjDFrLl03J4YIKJyv+0QhaAhQ1zZ9iTE+NVQyp5EfThA7JBKjGodh6382lPMJLPC2Z+jvDXaP33cDKCiOZdNVVV52Wl7Cnsp9aJIfIVZMciNtPR8r/NaY6HA78fv9Xj/Ql7VRe6LfFDbRFJDCe2QwYzortZtT58+ezbt26UQc/pV0aWU0dZ3Z4hrWls4pRlyxZAsCmTZvw+XyjBn5m6+0/a+4cM8tml/CLy6r4N9TSNqgREoRVAAAAJXRFWHRkYXRlOmNyZWF0ZQAyMDI0LTExLTExVDE2OjQ2OjA1KzAwOjAwGWMG9wAAACV0RVh0ZGF0ZTptb2RpZnkAMjAyNC0xMS0xMVQxNjo0NjowNSswMDowMGg+vksAAAAZdEVYdFNvZnR3YXJlAHd3dy5pbmtzY2FwZS5vcmeb7jwaAAAAAElFTkSuQmCC" alt="Open In Colab"/>
</a>

# Intro à GeoPandas

GeoPandas est une extension du module Pandas et rajoute la prise en charge de données géospatiales. La structure de données principale de GeoPandas est le **GeoDataFrame**. C'est un **DataFrame** de Pandas, mais qui supporte des colonnes géométriques. En fait, la géométrie est traitée simplement comme une colonne supplémentaire du DataFrame.

Chaque colonne est représentée par une GeoSeries, qui est l'équivalent GeoPandas des Series de Pandas. Donc un GeoDataFrame est simplement une collection de GeoSeries. Une GeoSeries peut contenir des données numériques, textuelles etc. comme dans Pandas, mais en plus aussi des géométries.

Chaque GeoDataFrame peut contenir en principe plusieurs colonnes géométriques, ce qui les différencie d'une couche vectorielle traditionnelle d'un SIG.

Toutes les opérations qui peuvent être faites sur un DataFrame de Pandas peuvent aussi être faites sur un GeoDataFrame.

## Objectifs

Dans ce tutoriel, nous allons explorer l'utilisation de GeoPandas pour manipuler des données géospatiales. GeoPandas est une extension de Pandas, et si vous êtes déjà familier avec Pandas, apprendre GeoPandas sera une transition naturelle. GeoPandas ajoute des capacités géospatiales aux DataFrames de Pandas, ce qui en fait un outil puissant pour analyser des données géographiques.

Nous allons découvrir les possibilités de GeoPandas à l'aide de deux exemples:

- Manipulation des données géospatiales des pays du monde : lecture, jointure, calculs géographiques et analyse statistique.
- Analyse géospatiale des données OpenStreetMap en Estonie : calcul des distances, agrégation des données, et statistiques géographiques.

## Partie 1: Indice démographique des pays du monde

Dans cet exemple, nous allons calculer un indice démographique pour les pays du monde, à l'aide des données de la Banque mondiale et des géométries de [Natural Earth](https://naturalearthdata.com).

Avant de commencer, nous devons obtenir les données nécessaires à l'analyse. Nous faisons cela avec le module `pooch`. Il faut d'abord s'assurer qu'il est installé:

In [ ]:
!pip install pooch

Les données sont prêtes à être téléchargées depuis l'URL suivante, sous forme d'une archive ZIP: [https://files.swisscarto.ch/geoinf1/demographie-monde.zip](https://files.swisscarto.ch/geoinf1/demographie-monde.zip). Nous pouvons le télécharger directement avec Pooch:

In [ ]:
import pooch

In [ ]:
data_files = pooch.retrieve(
    url = 'https://files.swisscarto.ch/geoinf1/demographie-monde.zip',
    known_hash = None,
    processor = pooch.Unzip()
)

Cette opération a permis de télécharger le fichier dans un dossier temporaire sur le disque dur et de le décompresser. `data_files` contient maintenant une liste avec tous les fichiers contenus dans l'archive ZIP.

### Lecture du fichier Shape des pays du monde

Nous allons maintenant charger le fichier Shape des pays du monde à l'aide de GeoPandas. Assurons-nous d'abord que GeoPandas est installé:

In [ ]:
!pip install geopandas

Nous pouvons import GeoPandas, nous utilisons le nom court `gpd` pour nommer le module:

In [ ]:
import geopandas as gpd

Nous devons maintenant obtenir le chemin d'accès du fichier Shape. L'archive ZIP contient un seul fichier Shape, nous pouvons donc filtrer les fichiers à l'aide de l'extension:

In [ ]:
pays_shp_path = [df for df in data_files if df.endswith('.shp')][0]
pays_shp_path

Le chargement du fichier Shape se fait simplement avec la fonction `read_file`:

In [ ]:
pays = gpd.read_file(pays_shp_path)

Inspecter le contenu du fichier:

In [ ]:
pays.plot()

### Lecture du fichier des données

Les données se trouvent dans un fichier TSV qui était à l'intérieur de l'archive ZIP. Nous pouvons obtenir le chemin d'accès comme avant pour le fichier Shape, sauf que nous cherchons pour l'extension `.tsv`:

In [ ]:
wb_data_path = [df for df in data_files if df.endswith('.tsv')][0]
wb_data_path

Nous pouvons lire le fichier avec Pandas. Il faut l'importer avant:

In [ ]:
import pandas as pd

In [ ]:
wb_data = pd.read_csv(wb_data_path, sep='\t')
print(wb_data.head())

Nous pouvons maintenant faire une jointure par attribut des données aux DataFrame des pays. Pour ceci, nous devons identifier deux colonnes qui contiennent la même information.

Nous pouvons inspecter les colonnes des deux DataFrames:

In [ ]:
pays.columns.tolist()

In [ ]:
wb_data.columns.tolist()

Nous allons utiliser la colonne `WB_A3` du DataFrame des pays, et la colonne `Country Code` du DataFrame des données de la Banque mondiale.

Nous pouvons maintenant faire la jointure avec la fonction `merge`:

In [ ]:
pays_data = pays.merge(wb_data, left_on='WB_A3', right_on='Country Code', how='left')

### Calculer un indicateur d'âge

Nous allons calculer un indicateur d’âge, défini comme le rapport entre la population âgée (65 ans et plus) et la population jeune (0-14 ans), ou plus extactement le nombre de personnes âgées (65 ans et plus) pour 100 personnes jeunes (0 à 14 ans).

Le calcul est assez simple une fois que l'on a trouvé les noms des colonnes. Vu que les noms des colonnes ne sont vraiment pas simples, nous le gardons dans une variable:

In [ ]:
indic_vieux = '2023 [YR2023] - Population ages 65 and above, total [SP.POP.65UP.TO]'
indic_jeunes = '2023 [YR2023] - Population ages 0-14, total [SP.POP.0014.TO]'

Avant de procéder au calcul, nous devons nous assurer que les valeurs sont numériques et que les valeurs non numériques sont des `NaN` (avec `errors = 'coerce'`):

In [ ]:
pays_data[indic_vieux] = pd.to_numeric(pays_data[indic_vieux], errors='coerce')
pays_data[indic_jeunes] = pd.to_numeric(pays_data[indic_jeunes], errors='coerce')

Ensuite le calcul en tant que tel:

In [ ]:
pays_data['indic_age'] = pays_data[indic_vieux] / pays_data[indic_jeunes] * 100

Vérifier le calcul:

In [ ]:
pays_data['indic_age']

### Créer des classes et faire le décompte des personnes

Nous allons maintenant diviser les pays en 5 classes selon l'indicateur d'âge. chaque classe aura à peu près le même nombre de pays. Nous utilisons la fonction `qcut` de Pandas pour cela. Consultez la documentation Pandas pour en savoir plus:

In [ ]:
pays_data['indic_age_cls'] = pd.qcut(pays_data['indic_age'], q=5, labels=False)

Et inspecter le résultat:

In [ ]:
print(pays_data[['NAME', 'indic_age_cls']].head())

Regardons le résultat sur une carte rapide:

In [ ]:
pays_data.plot(
    column = 'indic_age_cls',
    cmap = 'viridis',
    legend = True,
    figsize = (16, 12)
)

Nous avons certainement vu des meilleures cartes... mais elle fait l'affaire pour le moment. L'indicateur d'âge représente le ratio vieux / jeunes, et plus l'indicateur est élevé, plus il y a de personnes âgées par rapport aux jeunes. Il représente donc une facette du veillissement de la population, avec surtout les pays occidentaux ayant une valeur élevée.

Nous pouvons maintenant compter combien de personnes appartiennent à quelle classe de l'indicateur d'âge. Cette opération d'agrégation s'appelle un **«group by»** et on groupe toutes les entités avec la même valeur. Ensuite, on peut calculer des statistiques sur ces groupes, en l'occurrence la somme:

In [ ]:
col_pop = '2023 [YR2023] - Population, total [SP.POP.TOTL]'
pays_data[col_pop] = pd.to_numeric(pays_data[col_pop], errors='coerce')
pop_par_classe = pays_data.groupby('indic_age_cls')[col_pop].sum()

In [ ]:
pop_par_classe

Nous pouvons maintenant faire un graphique avec Matplotlib:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
pop_par_classe.plot(kind="bar", color="darkblue", edgecolor="white")
plt.title("Population par classe du ratio vieux / jeunes")
plt.xlabel("Class du ratio vieux / jeunes")
plt.ylabel("Population")
plt.show()

Pour finir cette partie, nous écrivons le jeu de données final dans un fichier Parquet:

In [ ]:
pays_data.to_file('pays_data.gpkg', driver='GPKG', layer='pays_data') 

## Partie 2: Analyse des données OpenStreetMap d'Estonie

Les données OpenStreetMap contiennent beaucoup de données que nous pouvons exploiter pour différentes analyses. Nous allons montrer quelques exemples concrets dans cette partie.

Il faut noter que les données OpenStreetMap sont des données qui sont générées par les utilisateurs et elles ne sont pas partout complètes au même degré. Dans certaines régions, l'administration publique contribue également aux données. Ou certaines entreprises utilisent les données OpenStreetMap dans leurs produits et aident à compléter la base de données (p.ex. Facebook, Apple, Microsoft utilisent tous les données OSM dans leur cartes).

Par ailleurs, si vous constatez que les données sur OpenStreetMap ne sont pas complètes, n'hésitez pas à les compléter sur [osm.org](https://osm.org). Àprès connexion, il y a un bouton «Éditer» qui devient disponible...

### Les bancs publics d'Estonie

Pour commencer, nous pouvons télécharger les données OpenStreetMap d'Estonie depuis l'URL suivante: [https://files.swisscarto.ch/geoinf1/analyse-estonie.zip](https://files.swisscarto.ch/geoinf1/analyse-estonie.zip).

L'archive ZIP contient également des régions administratives d'Estonie. Ces données viennent de l'Office statistique d'Estonie et contiennent également une information sur la population par région.

Nous pouvons à nouveau utiliser Pooch pour télécharger l'archive et ensuite la décompresser:

In [ ]:
data_ee = pooch.retrieve(
    url = 'https://files.swisscarto.ch/geoinf1/analyse-estonie.zip',
    known_hash = None,
    processor = pooch.Unzip()
)

Nous cherchons le fichier avec les points d'intérêts. Nous allons extraire les bancs publics depuis ce jeu de données (oui, oui, cette information y est) pour ensuite faire le décompte.

Le fichier des points d'intérêts est un fichier Shape (avec extension `.shp` et qui contient le mot `pois`:

In [ ]:
pois_shp_path = [f for f in data_ee if f.endswith('.shp') and 'pois' in f][0]
pois_shp_path

On peut maintenant lire le fichier avec GeoPandas:

In [ ]:
pois_ee = gpd.read_file(pois_shp_path)
print(pois_ee.head())

Les bancs publics ont la valeur `bench` pour la colonne `fclass`. On peut les extraire avec une expression un peu bizarre mais assez simple au final:

In [ ]:
bancs = pois_ee[pois_ee['fclass'] == 'bench']
print(bancs.head())

Nous pouvons décomposer un peu cette ligne de code. La partie `pois_ee['fclass']` est la colonne qui contient toutes les valeurs de l'attribut `fclass`:

In [ ]:
pois_ee['fclass']

On peut demander à Pandas si ces valeurs sont égales à `bench`:

In [ ]:
pois_ee['fclass'] == 'bench'

Ceci évalue cette comparaison pour chaque entité dans le DataFrame `pois_ee`, et retourne soit `True` soit `False`. Nous pouvons utiliser ce mécanisme pour filtrer les entités du DataFrame, en passant une sorte de liste avec des `True` et `False`. Les entités avec `True` sont retenues, les autres écartées.

On peut même enregistrer la condition dans une variable et écrire la même instruction que plus haut en deux lignes:

In [ ]:
critere = pois_ee['fclass'] == 'bench'
bancs = pois_ee[critere]
print(bancs.head())

### Les régions d'Estonie

Nous pouvons maintenant charger la couche des régions administratives depuis le Geopackage:

In [ ]:
regions_ee_path = [f for f in data_ee if f.endswith('.gpkg')][0]
regions_ee_path

Les fichiers Geopackage peuvent contenir plusieurs couches vectorielles. Si le Geopackage n'a qu'une seule couche, GeoPandas pourra la lire sans indiquer le nom. Mais avec plus d'une couche, il faudra spécifier le nom. Seulement, comment connaître le nom de la couche? Pour cela, nous pouvons utiliser un autre module Python: `pyogrio` (nous avons déjà rencontré `ogr` dans `ogr2ogr`, c'est bien le même). `pyogrio` est utilisé par GeoPandas et donc déjà disponible sur notre système. Il suffit de l'importer:

In [ ]:
import pyogrio

Ensuite on peut afficher la liste des couches à l'intérieur du GeoPackage:

In [ ]:
pyogrio.list_layers(regions_ee_path)

In [ ]:
regions = gpd.read_file(regions_ee_path,  layer='boundaries')
print(regions.head())

### Le nombre de bancs publics par région et par habitant

Combien de bancs publics il y a par région? Pour cela, il faut savoir dans quelle région se trouve chacun des bancs. Dans un SIG, une telle opération se fait à l'aide d'une **jointure spatiale**. Et avec GeoPandas, une jointure spatiale n'est pas compliquée, la fonction `sjoin` s'en occupe. Voici comment nous pouvons joindre les informations sur les régions à la couche des bancs:

In [ ]:
bancs_avec_region = gpd.sjoin(bancs, regions, how="inner", predicate="within")
bancs_avec_region.head()

L'information sur la région administrative est du coup dans `name_right`. Il suffit de compter le nombre de fois que chaque région apparaît dans la liste et on connaît le nombre de bancs dans chaque région...

Techniquement, il s'agit ici d'une agrégation de type **«group by»** (on regroupe les bancs selon les régions), et ensuite on fait le décompte (le nombre):

In [ ]:
bancs_decompte = bancs_avec_region.groupby('name_right').size()
bancs_decompte

Cette combinaison entre jointure spatiale et agrégation de type «group by» est très fréquent dans les analyses SIG. QGIS possède d'ailleurs un outil qui combine les deux étapes ensemble, il s'agit de l'outil *«Joindre les attributs par localisation (résumé)»*, qui est une variante de la jointure spatiale.

Par contre, nous devons encore joindre notre décompte à la couche des régions, dans une nouvelle colonne:

In [ ]:
regions['n_bancs'] = regions['name'].map(bancs_decompte)
regions.head()

Nous pouvons maintenant calculer le nombre de bancs par 1000 habitants et en faire une carte rapide:

In [ ]:
regions['n_bancs_par_1k'] = regions['n_bancs'] / (regions['population'] / 1000)
regions[['name', 'n_bancs', 'n_bancs_par_1k']].head()

In [ ]:
regions.plot(
    column = 'n_bancs_par_1k',
    cmap = 'viridis',
    legend = True
)

Avec un peu de chance nous avons trouvé les régions les plus touristiques d'Estonie et nous pouvons planifier nos vacances. Autrement, il y aura au moins assez de bancs pour se reposer.

### La longueur du rail par région

Nous pouvons aussi déterminer la longueur du rail par région à l'aide des données OSM. Commençons par charger la couche des trains:

In [ ]:
rail_shp_path = [f for f in data_ee if f.endswith('.shp') and 'railways' in f][0]
rail_shp_path

Lire les données:

In [ ]:
rails = gpd.read_file(rail_shp_path)

Pour déterminer la région pour les lignes des trains, il faut découper les lignes aux limites des régions. L'opération SIG d'intersection permet de faire cela et combine en même temps les attributs des deux couches impliquées. Dans GeoPandas, nous utilisons la fonction `overlay`:

In [ ]:
rails_par_region = gdp.overlay(rails, regions, how="intersection")

Nous pouvons encore ajouter la longueur des trains sur la base des lignes. Notez que cette opération dépend du système de coordonnées. Dans notre cas, nous avons un jeu de données en WGS84. Un système de coordonnées géographique en latitude/longitude ne se prête pas à un tel calcul. Nous allons projeter la couche d'abord dans un système de coordonnées UTM (la zone 35N correspond à l'Estonie, le code EPSG est 32635):

In [ ]:
rails_par_region_utm = rails_par_region.to_crs(epsg=32635)

Pour les opérations sur les géométries, on peut passer par l'attribut `geometry` du DataFrame GeoPandas. **Ceci nous permet de calculer des choses comme la longueur, la superficie ou le centroïde.**

In [ ]:
rails_par_region_utm['longueur'] = rails_par_region_utm.geometry.length

Et nous pouvons à nouveau faire une agrégation de type *«group by»* et calculer la somme. Il faut encore connaître le nom de la colonne qui contient le nom de la région:

In [ ]:
rails_par_region_utm.head()

In [ ]:
rail_longueurs = rails_par_region_utm.groupby('name_2')['longueur'].sum()
rail_longueurs.head()

Et nous pouvons ajouter l'information au GeoDataFrame sous forme d'une nouvelle colonne:

In [ ]:
regions["rail_longueur"] = regions["name"].map(rail_longueurs)
regions.head()

Par ailleurs, l'unité de la longueur est la même que celle du système de coordonnées. Pour UTM, ce sont des mètres.